## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-27-13-15-42 +0000,nyt,Anthropic Says It Cannot ‘Accede’ to Pentagon ...,https://www.nytimes.com/2026/02/26/technology/...
1,2026-02-27-13-12-58 +0000,nyt,Paramount Is Set to Win Warner Bros. Now Comes...,https://www.nytimes.com/2026/02/27/business/de...
2,2026-02-27-13-10-45 +0000,cbc,U.S. says embassy staff in Israel who want to ...,https://www.cbc.ca/news/world/united-states-is...
3,2026-02-27-13-08-03 +0000,bbc,What we know after latest escalation in Pakist...,https://www.bbc.com/news/articles/cj32zx48xvxo...
4,2026-02-27-13-05-27 +0000,bbc,SSE Airtricity to reduce gas prices by 8% from...,https://www.bbc.com/news/articles/clyg21wgejvo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2572/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
76,trump,54
710,epstein,19
110,new,18
52,war,17
432,ai,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
215,2026-02-27-00-03-07 +0000,nypost,House Dems to force vote to block Trump from w...,https://nypost.com/2026/02/26/us-news/house-de...,112
202,2026-02-27-00-53-44 +0000,nypost,Paramount Skydance victory in Warner Bros. Dis...,https://nypost.com/2026/02/26/media/paramount-...,109
326,2026-02-26-19-24-19 +0000,nypost,DOJ probes whether any Epstein files about unp...,https://nypost.com/2026/02/26/us-news/doj-prob...,107
292,2026-02-26-21-28-00 +0000,wsj,Oil lobbyists have made the case to Trump offi...,https://www.wsj.com/politics/policy/big-oil-is...,103
305,2026-02-26-20-53-08 +0000,nypost,Columbia student detained by ICE released afte...,https://nypost.com/2026/02/26/us-news/columbia...,102


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
215,112,2026-02-27-00-03-07 +0000,nypost,House Dems to force vote to block Trump from w...,https://nypost.com/2026/02/26/us-news/house-de...
268,85,2026-02-26-22-06-57 +0000,nypost,Paramount Skydance comes out on top of bidding...,https://nypost.com/2026/02/26/business/warner-...
180,83,2026-02-27-02-13-00 +0000,wsj,Hillary Clinton said that she had no knowledge...,https://www.wsj.com/politics/republicans-press...
419,66,2026-02-26-14-25-08 +0000,bbc,A New York City snowball fight gone wrong leav...,https://www.bbc.com/news/articles/cwyxlgkj0k3o...
356,60,2026-02-26-18-08-06 +0000,nypost,Mortgage rates dip below 6% for first time sin...,https://nypost.com/2026/02/26/business/mortgag...
292,49,2026-02-26-21-28-00 +0000,wsj,Oil lobbyists have made the case to Trump offi...,https://www.wsj.com/politics/policy/big-oil-is...
85,48,2026-02-27-10-30-00 +0000,wsj,"Neither theory, history nor the latest data su...",https://www.wsj.com/economy/jobs/tech-has-neve...
134,47,2026-02-27-06-07-00 +0000,wsj,The Pentagon said it shot down a “seemingly th...,https://www.wsj.com/politics/national-security...
27,46,2026-02-27-12-02-50 +0000,nypost,Huckabee tells Israel embassy staff to leave ‘...,https://nypost.com/2026/02/27/us-news/mike-huc...
138,46,2026-02-27-06-04-07 +0000,nypost,Notorious pedophile slapped with new charges j...,https://nypost.com/2026/02/27/us-news/notoriou...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
